In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import matplotlib 
# from rdkit import Chem
import os

from torch_geometric.loader import DataLoader
import torch
from utils_data import TestbedDataset
from utils_preproc import *

/home/conghao001/miniconda3/envs/gnndrug/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
branch = '069'
path = 'root_folder/root_' + branch

In [3]:
cell_dict, cell_feature, genes = save_gene_expr_matrix_X(None)
# cell_dict

In [4]:
cell_feature.shape

(700, 13142)

In [5]:
genes = genes.values
gene_list = np.array([g.split(' (')[0] for g in genes])

In [6]:
gene_list.shape

(13142,)

In [7]:
cell_feature[0].shape

(13142,)

In [8]:
ge_df = pd.DataFrame(cell_feature, index=list(cell_dict.keys()), columns=gene_list)
ge_df

,KRT19,SPARC,C19orf33,VIM,UCHL1,TGFBI,KRT7,FN1,KRT8,FXYD3,...,RPEL1,ECT2L,HNRNPCL1,KIF4B,FAM186A,SPDYE3,PRH2,CAGE1,PPP1R2B,AC090517.4
687455,10.970609,0.739848,8.044886,1.748461,1.384050,3.190615,11.812258,3.427606,10.431393,9.470191,...,0.189034,0.097611,0.163499,0.097611,0.014355,0.150560,0.070389,0.084064,0.111031,0.163499
1659929,9.360715,2.632268,7.482203,3.140779,1.150560,5.874059,2.060047,3.307429,11.374116,8.630267,...,0.124328,0.070389,0.056584,0.042644,0.014355,0.070389,0.028569,0.056584,0.056584,0.056584
684681,4.431623,0.823749,6.802839,1.475085,5.741197,6.598574,9.222867,4.015248,9.051019,7.455163,...,0.056584,0.042644,0.014355,0.070389,0.070389,0.056584,0.000000,0.070389,0.042644,0.070389
906836,2.000000,1.124328,0.956057,9.580485,1.304511,2.049631,1.432959,2.879706,3.798051,0.704872,...,0.189034,0.028569,0.613532,0.298658,0.137504,0.201634,0.097611,0.163499,0.176323,0.056584
906791,10.287712,1.327687,9.001239,1.464668,0.411426,1.084064,4.766595,8.755455,11.672779,3.636915,...,0.124328,0.137504,0.214125,0.042644,0.214125,0.070389,0.056584,0.042644,0.163499,0.070389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240218,9.385560,2.090853,8.680887,4.865919,4.989139,8.011898,10.982352,8.619340,10.996403,7.676803,...,0.189034,0.137504,0.056584,0.028569,0.084064,0.150560,0.286881,0.028569,0.084064,0.042644
908441,1.427606,5.685099,0.000000,9.353566,7.783522,2.292782,0.176323,5.436961,5.659354,0.014355,...,0.014355,0.042644,0.250962,0.070389,0.000000,0.042644,0.028569,0.201634,0.084064,0.000000
906798,3.737687,1.042644,5.580447,9.206355,9.000056,2.350497,9.889413,6.465322,10.390491,0.782409,...,0.084064,0.042644,0.250962,0.163499,0.056584,0.084064,0.097611,0.163499,0.124328,0.042644
905974,1.007196,7.974529,0.422233,11.202675,0.333424,6.248876,1.521051,7.721373,6.010108,0.495695,...,0.084064,0.042644,0.084064,0.084064,0.056584,0.042644,0.111031,0.056584,0.028569,0.084064


In [9]:
# tmp = []
iqr = ge_df.quantile(0.75) - ge_df.quantile(0.25)
iqr

KRT19         8.070962
SPARC         7.018246
C19orf33      7.547200
VIM           7.332947
UCHL1         7.717478
                ...   
SPDYE3        0.134696
PRH2          0.093976
CAGE1         0.081684
PPP1R2B       0.093976
AC090517.4    0.094859
Length: 13142, dtype: float64

In [10]:
lim = np.abs((ge_df - ge_df.median()) / iqr) < 2.22
filtered_ge_df = ge_df.where(lim, np.nan)
filtered_ge_df

,KRT19,SPARC,C19orf33,VIM,UCHL1,TGFBI,KRT7,FN1,KRT8,FXYD3,...,RPEL1,ECT2L,HNRNPCL1,KIF4B,FAM186A,SPDYE3,PRH2,CAGE1,PPP1R2B,AC090517.4
687455,10.970609,0.739848,8.044886,1.748461,1.384050,3.190615,11.812258,3.427606,10.431393,9.470191,...,0.189034,0.097611,0.163499,0.097611,0.014355,0.150560,0.070389,0.084064,0.111031,0.163499
1659929,9.360715,2.632268,7.482203,3.140779,1.150560,5.874059,2.060047,3.307429,11.374116,8.630267,...,0.124328,0.070389,0.056584,0.042644,0.014355,0.070389,0.028569,0.056584,0.056584,0.056584
684681,4.431623,0.823749,6.802839,1.475085,5.741197,6.598574,9.222867,4.015248,9.051019,7.455163,...,0.056584,0.042644,0.014355,0.070389,0.070389,0.056584,0.000000,0.070389,0.042644,0.070389
906836,2.000000,1.124328,0.956057,9.580485,1.304511,2.049631,1.432959,2.879706,3.798051,0.704872,...,0.189034,0.028569,NaN,0.298658,0.137504,0.201634,0.097611,0.163499,0.176323,0.056584
906791,10.287712,1.327687,9.001239,1.464668,0.411426,1.084064,4.766595,8.755455,11.672779,3.636915,...,0.124328,0.137504,0.214125,0.042644,0.214125,0.070389,0.056584,0.042644,0.163499,0.070389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240218,9.385560,2.090853,8.680887,4.865919,4.989139,8.011898,10.982352,8.619340,10.996403,7.676803,...,0.189034,0.137504,0.056584,0.028569,0.084064,0.150560,0.286881,0.028569,0.084064,0.042644
908441,1.427606,5.685099,0.000000,9.353566,7.783522,2.292782,0.176323,5.436961,5.659354,0.014355,...,0.014355,0.042644,0.250962,0.070389,0.000000,0.042644,0.028569,0.201634,0.084064,0.000000
906798,3.737687,1.042644,5.580447,9.206355,9.000056,2.350497,9.889413,6.465322,10.390491,0.782409,...,0.084064,0.042644,0.250962,0.163499,0.056584,0.084064,0.097611,0.163499,0.124328,0.042644
905974,1.007196,7.974529,0.422233,11.202675,0.333424,6.248876,1.521051,7.721373,6.010108,0.495695,...,0.084064,0.042644,0.084064,0.084064,0.056584,0.042644,0.111031,0.056584,0.028569,0.084064


In [11]:
filtered_ge_df.mean(axis=0, skipna=True)

KRT19         5.560794
SPARC         4.777188
C19orf33      4.739016
VIM           7.017893
UCHL1         4.985424
                ...   
SPDYE3        0.139647
PRH2          0.102763
CAGE1         0.080088
PPP1R2B       0.102346
AC090517.4    0.087510
Length: 13142, dtype: float64

In [12]:
gene_stats = ge_df.T
gene_stats['min'] = ge_df.min()
gene_stats['Q1'] = ge_df.quantile(0.25)
gene_stats['median'] = ge_df.quantile(0.5)
gene_stats['Q3'] = ge_df.quantile(0.75)
gene_stats['max'] = ge_df.max()
gene_stats['ori_mean'] = ge_df.mean(axis=0)
gene_stats['filtered_mean'] = filtered_ge_df.mean(axis=0, skipna=True)
gene_stats = gene_stats.iloc[:, -7:]
gene_stats

,min,Q1,median,Q3,max,ori_mean,filtered_mean
KRT19,0.0,1.550901,5.261095,9.621863,13.367551,5.560794,5.560794
SPARC,0.0,1.286845,3.569845,8.305090,14.309832,4.777188,4.777188
C19orf33,0.0,0.871844,5.091623,8.419044,11.047192,4.739016,4.739016
VIM,0.0,3.078078,7.953731,10.411025,13.274524,7.017893,7.017893
UCHL1,0.0,1.117695,4.571677,8.835173,11.485578,4.985424,4.985424
...,...,...,...,...,...,...,...
SPDYE3,0.0,0.066938,0.124328,0.201634,0.956057,0.147395,0.139647
PRH2,0.0,0.056584,0.097611,0.150560,1.000000,0.124633,0.102763
CAGE1,0.0,0.042644,0.070389,0.124328,0.739848,0.096042,0.080088
PPP1R2B,0.0,0.056584,0.097611,0.150560,1.416840,0.117566,0.102346


In [13]:
# gene_stats.to_csv('data/gene_statistics.csv')

In [13]:
gene_stats['filtered_mean'].values

array([5.56079396, 4.77718817, 4.73901584, ..., 0.08008788, 0.10234567,
       0.08751044])

In [14]:
drug_dict, drug_smile, smile_graph = load_drug_smile_X()
# drug_dict

# Explore salient genes of most sensitive drugs
## Find most sensitive drugs

In [15]:
ic50 = pd.read_csv('data/GDSC/PANCANCER_IC.csv', usecols=['Drug name', 'IC50'])
ic50_avg = ic50.groupby(['Drug name']).mean()
ic50_avg

,IC50
Drug name,
(5Z)-7-Oxozeaenol,0.817922
5-Fluorouracil,3.565578
681640,2.117400
A-443654,-0.739381
A-770041,2.083244
...,...
ZG-10,1.495344
ZM447439,2.216697
ZSTK474,1.016306


In [16]:
ic50_std = ic50.groupby(['Drug name']).std()
ic50_std

,IC50
Drug name,
(5Z)-7-Oxozeaenol,1.417219
5-Fluorouracil,1.767881
681640,1.101806
A-443654,1.379241
A-770041,1.791526
...,...
ZG-10,1.203072
ZM447439,1.264389
ZSTK474,1.707768


In [17]:
sensitivity = ic50_avg.merge(ic50_std, on=['Drug name'])
sensitivity = sensitivity.sort_values(by=['IC50_x'])
sensitivity = sensitivity.rename(columns={'IC50_x':'IC50_avg', 'IC50_y':'IC50_std'})
sensitivity

,IC50_avg,IC50_std
Drug name,,
Bortezomib,-5.647788,1.565107
Docetaxel,-4.901555,1.516071
Epothilone B,-4.861551,1.879182
SN-38,-4.240695,1.671931
Camptothecin,-4.060004,1.790315
...,...,...
Temozolomide,5.535896,0.805496
Cetuximab,5.981718,1.007772
DMOG,6.336191,1.443846


In [18]:
# sensitivity.to_csv('data/drug_sensitivity.csv')

In [19]:
sorted_ic50_avg = ic50_avg.sort_values(by=['IC50'])
sorted_ic50_avg.loc[sorted_ic50_avg['IC50']<=0]

,IC50
Drug name,
Bortezomib,-5.647788
Docetaxel,-4.901555
Epothilone B,-4.861551
SN-38,-4.240695
Camptothecin,-4.060004
Vinblastine,-4.049858
Thapsigargin,-3.830077
Vinorelbine,-3.797435
Sepantronium bromide,-3.685045


In [20]:
# selected_drugs = sorted_ic50_avg.loc[sorted_ic50_avg['IC50']<=0].index
# pd.DataFrame(selected_drugs.values).to_csv('data/sensitive_drugs.csv', index=None, header=None)

# pd.DataFrame(sorted_ic50_avg.index.values).to_csv('data/all_drugs.csv', index=None, header=None)

In [21]:
sorted_ic50_std = ic50_std.loc[sorted_ic50_avg.index]
# sorted_ic50_std.head(selected_drugs.shape[0])

In [22]:
# n_d = selected_drugs.shape[0]
# d_list = sorted_ic50_avg.index[:n_d].values

d_list = sorted_ic50_avg.index.values
d_list

array(['Bortezomib', 'Docetaxel', 'Epothilone B', 'SN-38', 'Camptothecin',
       'Vinblastine', 'Thapsigargin', 'Vinorelbine ',
       'Sepantronium bromide', 'Paclitaxel', 'Elesclomol', 'Omipalisib',
       'Bryostatin 1', 'Luminespib', 'Dacinostat', 'Daporinad',
       'GW843682X', 'Dactolisib', 'Rapamycin', 'Gemcitabine', 'CGP-60474',
       'Temsirolimus', 'BI-2536', 'Ispinesib Mesylate', 'Doxorubicin',
       'JW-7-52-1', 'THZ-2-102-1', 'QL-VIII-58', 'PD0325901',
       'Trametinib', 'Obatoclax Mesylate', 'rTRAIL', 'A-443654',
       'Mitomycin-C', 'Lestauritinib', 'SNX-2112', 'Tanespimycin',
       'TW 37', 'Methotrexate', 'AZD7762', 'CUDC-101', 'AR-42', 'JNK-9L',
       'MG-132', 'IOX2', 'AZD8055', 'Midostaurin', 'Shikonin', 'WZ3105',
       'Cytarabine', 'Pevonedistat', 'WIKI4', 'Belinostat', 'Tivozanib',
       'Dasatinib', 'Ponatinib', 'BMS-754807', 'Entinostat', 'NVP-TAE684',
       'Z-LLNle-CHO', '(5Z)-7-Oxozeaenol', 'CAY10603', 'Tozasertib',
       'Pictilisib', 'Vorinost

## load saliency score and ranks
select from drug list (sensitive ones or all)

In [23]:
from utils_decoding import *
# matplotlib.rc('xtick', labelsize=40) 
# matplotlib.rc('ytick', labelsize=40) 

In [55]:
model_name = 'SAGE'
ss_path = path + '/Saliency/IG/CellLine/' + model_name + '/iqr_mean_baseline'

drug_dict, sal_dict =  make_ss_dict(ss_path, type='drug')
norm_sal_dict = normalize_ss(sal_dict)
rank_dict = rank_ss(norm_sal_dict)

In [56]:
print(len(rank_dict['Lenalidomide']), len(norm_sal_dict), len(drug_dict))

13142 223 223


In [57]:
def select_dict(ori_dict, key_list):
    return {k:v for k, v in ori_dict.items() if k in key_list}

In [58]:
selected_sal_dict = select_dict(norm_sal_dict, d_list)
selected_drug_dict = select_dict(drug_dict, d_list)
selected_rank_dict = select_dict(rank_dict, d_list)

## Drug gene interaction

### first let's find all drug gene interaction pairs in our dataset
among all our drugs and all the genes
- 2400 pairs in total
- out of 3202 pairs from dgi database)

In [28]:
# interac = pd.read_csv('data/sensitive_drug_gene_interaction.tsv', sep='\t', usecols=['search_term', 'gene'])
interac = pd.read_csv('data/all_drug_gene_interaction.tsv', sep='\t', usecols=['search_term', 'gene'])

interac = interac.set_index('search_term')
interac

,gene
search_term,
BORTEZOMIB,PSMC2
BORTEZOMIB,GATA2
BORTEZOMIB,PSMD4P1
BORTEZOMIB,PSMD11
BORTEZOMIB,PSMB1
...,...
CETUXIMAB,KDR
CETUXIMAB,IDH1
PHENFORMIN,STK11


In [59]:
l_interac_all = []
for k, v in rank_dict.items():
    ranked_genes = gene_list[v]
    sal_score = norm_sal_dict[k].reshape(-1)
    ranked_ss = sal_score[v]
    if k.upper() not in interac.index:
        continue
    drug_gene = interac.loc[k.upper()].values
    
#     for g in ranked_genes:
    for i in range(len(ranked_genes)):
        g = ranked_genes[i]
        ss = ranked_ss[i]
        if g in drug_gene:
#             print(k, g)
            l_interac_all.append([k, g, i, ss])
            
interac_all_df = pd.DataFrame(l_interac_all, columns=['drug', 'gene', 'rank', 'sal_score']).set_index('drug')
interac_all_df

,gene,rank,sal_score
drug,,,
Camptothecin,GSTP1,126,0.514342
Camptothecin,VDR,301,0.438385
Camptothecin,BDNF,833,0.357178
Camptothecin,TGM2,970,0.343758
Camptothecin,MGMT,2306,0.241725
...,...,...,...
Belinostat,HDAC2,9892,0.025143
Belinostat,HDAC8,9906,0.025064
Belinostat,FBXW7,10963,0.018438


In [60]:
# interac_all_df.to_csv('root_folder/root_069/Saliency/IG/CellLine/GAT/iqr_mean_baseline_existing_dgi.csv')
interac_all_df.to_csv(path + '/Saliency/IG/CellLine/' + model_name + '/iqr_mean_baseline_existing_dgi.csv')

In [29]:
# top_n = 100
# for k, v in selected_rank_dict.items():
#     ranked_genes = gene_list[v][:top_n]
#     print(k, ranked_genes)

In [21]:
list(set(interac.index.values))

['ALECTINIB',
 'PELITINIB',
 'TAMOXIFEN',
 'AMUVATINIB',
 'PF-562271',
 'TEMOZOLOMIDE',
 'CAMPTOTHECIN',
 'TOZASERTIB',
 'PI-103',
 'BICALUTAMIDE',
 'AVAGACESTAT',
 'IMATINIB',
 'Y-39983',
 'FEDRATINIB',
 'CYTARABINE',
 'S-TRITYL-L-CYSTEINE',
 'BMS-754807',
 'LINIFANIB',
 'OMIPALISIB',
 'LUMINESPIB',
 'ENTINOSTAT',
 'DACINOSTAT',
 'BMS-345541',
 'SELUMETINIB',
 'SERDEMETAN',
 'ZIBOTENTAN',
 'BI-2536',
 'DOCETAXEL',
 'ERLOTINIB',
 'BORTEZOMIB',
 'QUIZARTINIB',
 'TUBASTATIN A',
 'RUXOLITINIB',
 'CUDC-101',
 'TIPIFARNIB',
 'SARACATINIB',
 'TRAMETINIB',
 'MOTESANIB',
 'SB590885',
 'PALBOCICLIB',
 'CISPLATIN',
 'CABOZANTINIB',
 'GSK690693',
 'MG-132',
 'LAPATINIB',
 'SN-38',
 'EMBELIN',
 'PAZOPANIB',
 'DASATINIB',
 'TANESPIMYCIN',
 '5-FLUOROURACIL',
 'BOSUTINIB',
 'GEMCITABINE',
 'MK-2206',
 'BLEOMYCIN',
 'ENZASTAURIN',
 'VELIPARIB',
 'DORAMAPIMOD',
 'T0901317',
 'THAPSIGARGIN',
 'OBATOCLAX MESYLATE',
 'NAVITOCLAX',
 'VORINOSTAT',
 'AZD6482',
 'CGP-60474',
 'TRETINOIN',
 'IDELALISIB',
 'BRY

In [25]:
top_n = 13143
for k, v in selected_rank_dict.items():
    ranked_genes = gene_list[v][:top_n]
    if k.upper() not in interac.index:
        continue
    drug_gene = interac.loc[k.upper()].values
    
    for g in ranked_genes:
        if g in drug_gene:
            print(k, g)
#     print(k, ranked_genes)

Camptothecin GSTP1
Camptothecin TGM2
Camptothecin COL18A1
Camptothecin THBS1
SN-38 BCHE
SN-38 CES1
SN-38 ABCC2
Axitinib PDGFRB
Axitinib PDGFRA
Pazopanib HLA-B
Pazopanib MET
Pazopanib FGFR1
Pazopanib FYN
Pazopanib FGFR2
Pazopanib PDGFRB
Pazopanib PDGFRA
NSC-87877 ADM
NSC-87877 PTPN6
Bicalutamide FST
Etoposide GSTP1
Etoposide GDF15
Etoposide FGFR1
Etoposide EGFR
Etoposide IGF2
PF-562271 MET
PF-562271 EPHA2
PF-562271 FGFR1
PF-562271 AXL
PF-562271 FYN
PF-562271 MST1R
PF-562271 PDGFRB
PF-562271 BTK
Selumetinib MET
Selumetinib FGFR1
Selumetinib EGFR
Selumetinib ERBB3
Selumetinib CDH1
Selumetinib FGFR2
Dactolisib EGFR
Dactolisib MET
AZD8055 FGFR1
Cytarabine GSTP1
Cytarabine BMP7
Cytarabine GATA3
Cytarabine CDA
Pictilisib FGFR2
Doramapimod EGFR
Doramapimod PDGFRB
Amuvatinib MET
Amuvatinib PDGFRB
Amuvatinib PDGFRA
Midostaurin FGFR1
Midostaurin PDGFRB
Midostaurin PDGFRA
MK-2206 EGFR
Ponatinib EGFR
Ponatinib FGFR1
Ponatinib MET
Ponatinib FGFR4
Ponatinib FGFR2
Ponatinib PDGFRB
Ponatinib PDGFRA
Ola

## Visualization

In [30]:
def draw_selected_drugs(rank_dict, sal_dict, gene_list, top_n=100):
    r, c = len(sal_dict), 1
    fig = plt.figure(1, figsize=[len(sal_dict)*15, top_n])

    i = 0
    for key in sal_dict.keys():
        i += 1
        rnk = rank_dict[key]
        sal_score = sal_dict[key].reshape(-1)
        ranked_ss = sal_score[rnk]
        ranked_genes = gene_list[rnk]
        
        values = ranked_ss[:top_n]
        index = ranked_genes[:top_n]
        
        plt.subplot(r, c, i)
#         sns.set(font_scale=10)
#         plt.title(key, fontdict={'fontsize': 30})
        res = sns.heatmap(values.reshape(1, -1), annot=True, annot_kws={'fontsize':50, 'rotation':"vertical",'verticalalignment':"center"}, cmap="YlGnBu")
        res.set_xticklabels(index, fontdict={'fontsize':50}, rotation=45)
        res.set_yticklabels([key], fontdict={'fontsize':50})
        
    plt.show()
    
    return fig

In [45]:
# fig = draw_selected_drugs(selected_rank_dict, selected_sal_dict, gene_list, 100)

In [32]:
save_path = path + '/SaliencyMap/IG/CellLine/' + model_name
fig.savefig(save_path + '/sensitive_drugs_gene_saliency.png')